# AMUSE: Community codes

In [ ]:
import numpy
numpy.random.seed(11)
from amuse.lab import *
from amuse.support.console import set_printing_strategy
set_printing_strategy(
    "custom",
    preferred_units=[units.MSun, units.parsec, units.Myr, units.kms],
    precision=6, prefix="", separator=" [", suffix="]",
)
converter = nbody_system.nbody_to_si(1 | units.parsec, 1000 | units.MSun)

Amuse contains many community codes, which can be found in amuse.community.
These are often codes that have been in use as standalone codes for a long time (e.g. Gadget2), but some are unique to AMUSE (e.g. ph4, a 4th order parallel Hermite N-body integrator with GPU support).

Each community code must be instantiated to start it, after which parameters can be set and particles added.

The code can then be instructed to evolve the particles to a specific time. Once it reaches this time, the code can be called again, or it can be stopped, removing it from memory and stopping the running process(es).

In [ ]:
test_sphere = new_plummer_model(1000, converter)
test_sphere.mass = new_salpeter_mass_distribution(1000, mass_min=0.3 | units.MSun)
def new_gravity(particles):
    gravity = ph4(converter, number_of_workers=1)
    gravity.parameters.epsilon_squared = (0.01 | units.parsec)**2
    gravity.particles.add_particles(particles)
    gravity_to_model = gravity.particles.new_channel_to(particles)
    return gravity, gravity_to_model
gravity, gravity_to_model = new_gravity(test_sphere)

print(test_sphere.center_of_mass())
print(gravity.particles.center_of_mass())
gravity.evolve_model(0.1 | units.Myr)
print(gravity.particles.center_of_mass())
print(test_sphere.center_of_mass())

gravity.stop()

Note that the original particles (`test_sphere`) were not modified, while those maintained by the code were (for performance reasons). Also, small numerical errors can arise at this point, the magnitude of which depends on the chosen converter units.

To synchronise the particle sets, AMUSE uses "channels". These can copy the required data when needed, e.g. when synchronising changes in particle properties to other codes.

In [ ]:
gravity, gravity_to_model = new_gravity(test_sphere)

print(gravity.particles.center_of_mass())
gravity.evolve_model(0.1 | units.Myr)
gravity_to_model.copy()
print(gravity.particles.center_of_mass())
print(test_sphere.center_of_mass())

gravity.stop()

## Combining codes: gravity and stellar evolution

In a simulation of a star cluster, we may want to combine several codes to address different parts of the problem:
- an N-body code for gravity,
- a stellar evolution code

In the simplest case, these interact only via the stellar mass, which is changed over time by the stellar evolution code and then updated in the gravity code.

In [ ]:
def new_evolution(particles):
    evolution = SSE()
    evolution.parameters.metallicity = 0.01
    evolution.particles.add_particles(particles)
    evolution_to_model = evolution.particles.new_channel_to(particles)
    return evolution, evolution_to_model

evolution, evolution_to_model = new_evolution(test_sphere)
gravity, gravity_to_model = new_gravity(test_sphere)
model_to_gravity = test_sphere.new_channel_to(gravity.particles)

time = gravity.model_time
end_time = 1 | units.Myr
while time < end_time:
    timestep = evolution.particles.time_step.min()
    gravity.evolve_model(time+timestep/2)
    evolution.evolve_model(time+timestep)
    evolution_to_model.copy()
    model_to_gravity.copy()
    gravity.evolve_model(time+timestep)
    time += timestep
    print("Now at time %s." % gravity.model_time, end=" ")
    print("The most massive star is now %s" % test_sphere.mass.max())
evolution.stop()
gravity.stop()

Note that the timestep is now set by the stellar evolution code, and is based on the evolution timescale of the stellar mass.